In [1]:
import pandas as pd
import stumpy
import numpy as np
import datetime as dt
import random
import math
import pickle
import sys

from statistics import mean
from tqdm.auto import tqdm
from multiprocessing import Pool

import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
'''
Collects random samples from trace with id2 and computes the matrix profile of class1 compared with class 2

Input: 
    trace1: packet traces from class 1
    id2: id number for class 2 
    num_traces: number of traces to select from class 2 (should be equal to class 1)
    shapelet_size: length of shapelets
    
Output:
    Matrix profile of trace1 compared with trace2
'''
def compare_profile(trace1, id2, num_traces, shapelet_size):
    
    trace2 = []
    while len(trace2) < shapelet_size:
        trace2 = random.sample(traces[id2], num_traces)
        trace2 = np.asarray([item for row in trace2 for item in row]).astype('float64')
    
    #print("self-profiles generated...")
    c1_c2 = stumpy.stump(trace1, shapelet_size, trace2, ignore_trivial=False)[:, 0].astype(float)
    c1_c2[c1_c2 == np.inf] = np.nan
    #print("Comparison profiles generated...")
    
    return c1_c2

'''
Compares a the matrix profile of a class trace with itself

Input: 
    trace: packet traces from class 1
    shapelet_size: length of shapelets
    
Output:
    Matrix profile of trace compared with trace
'''

def same_profile(trace, shapelet_size):
    
    c1_c1 = stumpy.stump(trace, shapelet_size)[:, 0].astype(float)
    c1_c1[c1_c1 == np.inf] = np.nan
    
    return c1_c1

'''
return indices of shapelet as one-hot encoded list
'''
def generate_shapelet(trace, diff, shapelet_size):
    
    idx = np.argmax(diff)
    shapelet = np.asarray([1 if idx <= i < idx + shapelet_size else 0 for i in range(len(trace))])
    
    return shapelet

'''
Compute shapelet of greatest overlaps
'''
def find_overlap(trace_i, shapelets_i, shapelet_size):
    #print(shapelets_i[0])
    
    merged_shapelets = np.sum(shapelets_i, axis=0)
    
    max_size = 0
    start = 0
    end = 0
    
    for i in range(0, len(merged_shapelets), shapelet_size):
        current_size = np.sum(merged_shapelets[i:i+shapelet_size])
        if current_size > max_size:
            max_size = current_size
            start = i
            end = i + shapelet_size
    
    return trace_i[start:end]

In [3]:
'''
Generates a set of 100 shapelets for each class in samples

Input:
    num_traces = Number of traces per class
    shapelet_size = Size of shapelets
    save: save results to file?
    filename: if save, name & location of output file

Output:
    list object containing shapelets for each class

'''

def generate_shapelets(num_traces, shapelet_size):
    shapelet_storage = []
    
    # loop over all classes (generate shapelet for each class)
    for i in tqdm(range(100)):
        
        # get num_traces samples from trace #i
        # while loop guarantees that the traces selected exceed shapelet size (or crashes will happen)
        trace_i = []
        while len(trace_i) < shapelet_size:
            trace_i = random.sample(traces[i], num_traces)
            trace_i = np.asarray([item for row in trace_i for item in row]).astype('float64')
        
        shapelets_i = np.zeros((100, len(trace_i)))
        #print(shapelets_i.shape)
        
        # generate profile of i compared with itself
        ci_ci = same_profile(trace_i, shapelet_size)
        
        # loop over every other class and generate a profile for each one
        for j in range(100):
            # don't compare i with itself 
            if i == j:
                continue

            # compute profile of i compared with j
            ci_cj = compare_profile(trace_i, j, num_traces, shapelet_size)

            # find largest value gap between other and i
            diff_ci = ci_cj - ci_ci
            
            # generate best shapelet for i compared to j and store it in list
            ci_shape = generate_shapelet(trace_i, diff_ci, shapelet_size)
            shapelets_i[j] = ci_shape
        
        # compare shapelets between all classes and return the one which has the most overlap
        # (i.e.) the shapelet that was chosen most between the 99 other classes
        best_shapelet = find_overlap(trace_i, shapelets_i, shapelet_size)
        # save to list
        shapelet_storage.append(best_shapelet)
    
    return shapelet_storage   

In [4]:
'''
Compute the minimum distance beteen data samples and shapelets
Input:
    data = list of individual packet traces
    shapelets = list of shapelets
Output:
    minimum distance between each sample in data compared with each sample in shapelet
    shape = (len(data),len(shapelets))
'''
def distance_to_shapelet(data, shapelets):
    #data = np.asarray(data)
    #print(len(data))
    
    # processed output data
    data_out = np.zeros((len(data),len(shapelets)))
    
    # loop over each sample in the dataset
    for i,sample in enumerate(tqdm(data)):
        shapelet_score = np.empty(len(shapelets))
        # for each shapelet, calculate distance and assign a score
        for j,shapelet in enumerate(shapelets):
            dist = stumpy.mass(shapelet, sample)
            shapelet_score[j] = dist.min()
        data_out[i] = shapelet_score
    
    return data_out

'''
Computes distances between input samples and shapelets, returns X for classifier
Also cleans data and ensures no random errors due to length, NaN, etc...
Underlying function that performs comparison is distance_to_shapelet
Selects data samples (with replacement)
note: some samples will always be bad so actual length of X is less

Input:
    num_traces = numner of traces to process
    save = save output to file
    filenames = tuple that represents (name of X file, name of y file)

Output:
    X values for classifier of shape (None, 100)
    y values for classifier of shape (None, )
'''

def process_traces(num_traces, shapelets, shapelet_size, save=True, filenames=("X.pkl","y.pkl")):
    X, y = [], []

    for i in range(num_traces):
        random_id = random.randrange(100)
        random_trace = random.choice(traces[random_id])
        X.append([random_trace])
        y.append(random_id)

    # process and remove useless entries (too short)
    X = [np.asarray(trace).astype('float64') for trace in X]
    X = [trace[~np.isnan(trace)] for trace in X]    
    removals = [i for i,x in enumerate(X) if len(x) < shapelet_size]
    for idx in removals:
        X[idx] = None
        y[idx] = None
    X = [trace for trace in X if trace is not None]
    y = [value for value in y if value is not None]

    # compute distance between input trace and shapelet arrays
    # return as new X

    X = distance_to_shapelet(X, shapelets)
    
    if save:
        with open(filenames[0], 'wb') as f:
            pickle.dump(X, f)

        with open(filenames[1], 'wb') as f:
            pickle.dump(y, f)
    
    return X, y

In [5]:
'''
Evaluate performance of sklearn classifier on data samples - 90/10 training testing split

Input:
    clf: sklearn classifier object
    X: x values
    y: y values
    topk: k values for evaluation metrics
Output:
    list of length topk with accuracy for testing data
'''

def classifier_performance(clf, X, y, topk=[1,3,5]):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    clf.fit(X_train, y_train)
    y_prob = clf.predict_proba(X_test)
    
    scores = []
    for k in topk:
        correct = 0
        for i in range(len(y_prob)):
            ind = np.argpartition(y_prob[i], -k)[-k:]
            if y_test[i] in ind:
                correct += 1
        scores.append(correct/len(y_prob))
    
    return scores

In [6]:
'''
Utility function for pipeline of evaluating different grid search parameters
Output: a new file located in ../results/param1-val1_param2-val2_param3-val3
        the file contains a pickled python object
        with the scores for top-1, top-3, and top-5 classifier accuracy
'''
# note: python multiprocessing is really annoying to work with
# function needs to be in a separate .py file which is imported
# and function can only have 1 argument
# list input which is immediately used for what would be the arguments
def evaluate_parameters(arr):
    
    global traces
    with open('../nonzero_traces.npy', 'rb') as f:
        traces = pickle.load(f)
    
    num_shapelet_samples = arr[0]
    shapelet_size = arr[1]
    num_clf_samples = arr[2]
    
    filename = '../results/shapelets/' + 'num=' + str(num_shapelet_samples) + 'size=' + str(shapelet_size)
    with open(filename, 'rb') as f:
        shapelets = pickle.load(f)
    
    print("Processing Traces...")
    X, y = process_traces(num_clf_samples, shapelets, shapelet_size, False)
    
    filename = '../results/data/X/' + 'num=' + str(num_shapelet_samples) + 'size=' + str(shapelet_size) + 'samples=' + str(num_clf_samples)
    
    with open(filename, 'wb') as f:
        pickle.dump(X, f)
        
    filename = '../results/data/y/' + 'num=' + str(num_shapelet_samples) + 'size=' + str(shapelet_size) + 'samples=' + str(num_clf_samples)
    
    with open(filename, 'wb') as f:
        pickle.dump(y, f)

In [15]:
# SETUP

global traces

with open('../nonzero_traces.npy', 'rb') as f:
    traces = pickle.load(f)

In [23]:
# PART 1
parameter_list = np.array(np.meshgrid(shapelet_samples_list, shapelet_size_list)).T.reshape(-1,2)

for i in range(18):
    shapelets = generate_shapelets(4, 500)
    
    filename = '../results/shapelets/' + 'num=' + str(i) + 'size=' + str(500)
    
    with open(filename, 'wb') as f:
        pickle.dump(shapelets, f)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [24]:
# PART 2
if __name__ == '__main__':
    
    global traces

    with open('../nonzero_traces.npy', 'rb') as f:
        traces = pickle.load(f)
    
    shapelet_samples_list = range(0, 18)
    shapelet_size_list = [500]
    clf_samples_list = [400000]

    from utils import evaluate_parameters
    
    parameter_list = np.array(np.meshgrid(shapelet_samples_list, shapelet_size_list, clf_samples_list)).T.reshape(-1,3)
    
    print(parameter_list)
    
    with Pool(6) as p:
        p.map(evaluate_parameters, parameter_list)

[[     0    500 400000]
 [     1    500 400000]
 [     2    500 400000]
 [     3    500 400000]
 [     4    500 400000]
 [     5    500 400000]
 [     6    500 400000]
 [     7    500 400000]
 [     8    500 400000]
 [     9    500 400000]
 [    10    500 400000]
 [    11    500 400000]
 [    12    500 400000]
 [    13    500 400000]
 [    14    500 400000]
 [    15    500 400000]
 [    16    500 400000]
 [    17    500 400000]]


  0%|          | 0/337426 [00:00<?, ?it/s]

Processing Traces...
Processing Traces...
Processing Traces...


  0%|          | 0/337625 [00:00<?, ?it/s], 12.75it/s]

Processing Traces...
Processing Traces...
Processing Traces...


...

  2%|▏         | 5860/337479 [06:46<6:54:40, 13.33it/s]

Processing Traces...


  2%|▏         | 6487/337479 [07:31<7:45:10, 11.86it/s]

Processing Traces...


100%|██████████| 337606/337606 [6:29:18<00:00, 14.45it/s]


In [25]:
## PART 3

parameter_list = np.array(np.meshgrid(shapelet_samples_list, shapelet_size_list, clf_samples_list)).T.reshape(-1,3)

print(parameter_list)

for parameters in parameter_list:
    
    filename = '../results/data/X/' + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1]) + 'samples=' + str(parameters[2])
    
    with open(filename, 'rb') as f:
        X = pickle.load(f)
        
    filename = '../results/data/y/' + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1]) + 'samples=' + str(parameters[2])
    
    with open(filename, 'rb') as f:
        y = pickle.load(f)
    
    clf = RandomForestClassifier()
    scores = classifier_performance(clf, X, y)
    
    print(scores)
    
    outfile_name = "../results/scores/" + 'num=' + str(parameters[0]) + 'size=' + str(parameters[1]) + 'samples=' + str(parameters[2])
    
    with open(outfile_name, 'wb') as f:
        pickle.dump(scores, f)

[[     0    500 400000]
 [     1    500 400000]
 [     2    500 400000]
 [     3    500 400000]
 [     4    500 400000]
 [     5    500 400000]
 [     6    500 400000]
 [     7    500 400000]
 [     8    500 400000]
 [     9    500 400000]
 [    10    500 400000]
 [    11    500 400000]
 [    12    500 400000]
 [    13    500 400000]
 [    14    500 400000]
 [    15    500 400000]
 [    16    500 400000]
 [    17    500 400000]]
[0.8069073783359497, 0.8779953200438375, 0.9073190959983413]
[0.7975283762558161, 0.8741072222386865, 0.9035948196663012]
[0.8094899380575561, 0.8818647935745828, 0.909486974304259]
[0.8069141215155375, 0.880084130698818, 0.9102704624226087]
[0.7933240529573794, 0.8681989159731066, 0.89992003080295]
[0.7953332542694497, 0.8693666982922201, 0.8993121442125237]
[0.8040794576512034, 0.8770241866248261, 0.9048225228692383]
[0.7896980461811723, 0.8627590290112492, 0.8915334517465956]
[0.8024640170585796, 0.8767695314813718, 0.9062666587691761]
[0.7989680021351679, 0